# Can ESG generate Alpha

In [1]:
import eikon as ek
import pandas as pd
import plotly.offline as offline
from plotly import graph_objs as go
from IPython.display import display, Markdown
offline.offline.init_notebook_mode(connected=True)

In [2]:
ek.set_app_key('-- YOUR EIKON KEY HERE --')

In [3]:
start_year = 2014
end_year = 2020
index = '.SPX'

### Get the performance data for the index benchmark.

In [4]:
sp,err = ek.get_data(index, fields=['TR.CLOSEPRICE.date', 'TR.CLOSEPRICE'], parameters={'SDate': '{}-01-01'.format(start_year), 'EDate': '{}-01-01'.format(end_year), 'Frq':'CQ'})
sp['Rebased'] = sp['Close Price'] * 100 / sp['Close Price'][0]
display(sp.head())
display('Count: {}'.format(len(sp)))

,Instrument,Date,Close Price,Rebased
0,.SPX,2014-03-31T00:00:00Z,1872.34,100.000000
1,.SPX,2014-06-30T00:00:00Z,1960.23,104.694126
2,.SPX,2014-09-30T00:00:00Z,1972.29,105.338240
3,.SPX,2014-12-31T00:00:00Z,2058.90,109.964002
4,.SPX,2015-03-31T00:00:00Z,2067.89,110.444150


'Count: 24'

In [5]:
def getSubset(dFrame):
    # filter out the companies with low ESG score at the begining year
    # meanScore = dFrame.mean()['ESG Score']
    dFrame = dFrame[dFrame['ESG Score'] > 70]
    return dFrame['Instrument'].tolist()

In [6]:
def getDataForYear(year):
    display('Getting data for: {}'.format(year))
    # get index constituents at the begining year
    df,err = ek.get_data('{}{}({}-01-01)'.format('0#', index, year), fields=['TR.TRESGScore'], parameters={'SDate': '{}-01-01'.format(year), 'Period': 'FY0'})
    # filter out the instruments based on ESG ratings
    subset = getSubset(df)
    # get the performance data for this subset
    df2,err = ek.get_data(subset, fields=['TR.CLOSEPRICE.date', 'TR.CLOSEPRICE'], parameters={'SDate': '{}-01-01'.format(year), 'EDate': '{}-01-01'.format(year + 1), 'Frq':'CQ'})
    df2 = df2.dropna()
    # consolidate the price data for instruments
    alphaList = []
    ser = df2['Date'].value_counts()
    for x in ser[ser > 5].index:
        ser = df2.loc[df2['Date'] == x].sum()
        alphaList.append({'Date': x, 'Close Price': ser['Close Price']})
    return alphaList


### Get the price performance of equal weighted, high ESG index constituents

In [7]:
df3 = pd.DataFrame()
for x in range(start_year, end_year):
    ret = getDataForYear(x)
    df3 = df3.append(ret)
df3 = df3.sort_values(by=['Date'])
df3 = df3.reset_index(drop=True)
df3['Rebased'] = df3['Close Price'] * 100 / df3['Close Price'][0]
display(df3.head())
display('Count: {}'.format(len(df3)))

'Getting data for: 2014'

'Getting data for: 2015'

'Getting data for: 2016'

'Getting data for: 2017'

'Getting data for: 2018'

'Getting data for: 2019'

,Close Price,Date,Rebased
0,6304.972105,2014-03-31T00:00:00Z,100.000000
1,6518.366506,2014-06-30T00:00:00Z,103.384542
2,6483.187610,2014-09-30T00:00:00Z,102.826587
3,6748.892885,2014-12-31T00:00:00Z,107.040805
4,7516.393813,2015-03-31T00:00:00Z,119.213752


'Count: 22'

### Plot the comparison

In [8]:
# plot the graphs together
res = [go.Scatter(x=sp['Date'], y=sp['Rebased'], name='Index', mode='lines'),
       go.Scatter(x=df3['Date'], y=df3['Rebased'], name='ESG List', mode='lines')]
lay = {'shapes': [
        {'type': 'line','x0': '{}-01-01'.format(start_year),'y0': 100,'x1': '{}-01-01'.format(end_year),'y1': 100,'line': {'width': 1},}
    ],
    'showlegend': True
}
fig = go.Figure(data=res, layout=lay)
fig.update_layout(title='Comparison')
fig.show()